In [1]:
#!/usr/bin/env python
# coding: utf-8


import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from mordred import Calculator, descriptors
calc = Calculator(descriptors, ignore_3D=True)

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, roc_auc_score, matthews_corrcoef, average_precision_score, confusion_matrix
from imblearn.metrics import geometric_mean_score
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline
from sklearn.metrics import roc_curve
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
from scipy.stats import randint, uniform
from sklearn.model_selection import StratifiedKFold
import math

from pandarallel import pandarallel
pandarallel.initialize()

import sys
sys.path.append('/home/ss2686/03_DICTrank')
import argparse
from scripts.evaluation_functions import evaluate_classifier, optimize_threshold_j_statistic


# Path where your data is stored
data_path = '../data/processed_binarised__splits/'

results = {}
held_out_results = []

def get_Mordred_columns_to_use():
    
        datasets = {}
        directory='../data/processed/'
        # Load datasets from given directory
        for foldername in os.listdir(directory):

            if not foldername.startswith('.'):  # Ignore folders starting with a dot

                print(foldername)
                file_path = os.path.join(directory, foldername, f"{foldername}_processed.csv.gz")

                if os.path.exists(file_path):
                    datasets[foldername] = pd.read_csv(file_path, compression='gzip')
                else:
                    print(f"No matching file found for folder: {foldername}")
        
        smiles_list = []
        
        for featuresets in ["sider", "DICTrank"]: 
    
            smiles_list.extend(datasets[featuresets].Standardized_SMILES.to_list())
            print(len(smiles_list))
        smiles_list = list(set(smiles_list))
        
        print(len(smiles_list))
        data = pd.DataFrame(smiles_list, columns=["Standardized_SMILES"])
        
        Ser_Mol_train = data['Standardized_SMILES'].apply(Chem.MolFromSmiles)
        Mordred_table_data = calc.pandas(Ser_Mol_train)
        Mordred_table_data = Mordred_table_data.astype('float')
        Mordred_table_data = Mordred_table_data.dropna(axis='columns')
        data_columns = Mordred_table_data.columns
        
        return(data_columns)
    
data_columns = get_Mordred_columns_to_use()




INFO: Pandarallel will run on 76 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
GeneExpressionSMILES
sider
MOA
CellPaintingSMILES
Cmax
DrugBank
DICTrank
1378
2613
1579


  0%|▏                                         | 7/1579 [00:00<02:24, 10.91it/s]

/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


  3%|█▍                                       | 55/1579 [00:01<04:30,  5.64it/s]

/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


...

 87%|██████████████████████████████████     | 1381/1579 [01:06<00:06, 28.37it/s]

/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 98%|██████████████████████████████████████▎| 1550/1579 [01:06<00:00, 58.88it/s]

/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|███████████████████████████████████████| 1579/1579 [01:09<00:00, 22.78it/s]


In [2]:
data_columns

Index(['ABC', 'ABCGG', 'nAcid', 'nBase', 'SpAbs_A', 'SpMax_A', 'SpDiam_A',
       'SpAD_A', 'SpMAD_A', 'LogEE_A',
       ...
       'SRW09', 'SRW10', 'TSRW10', 'MW', 'AMW', 'WPath', 'WPol', 'Zagreb1',
       'Zagreb2', 'mZagreb2'],
      dtype='object', length=1038)

In [25]:
import pickle

# Your existing code to get data_columns

# Save data_columns to a .pkl file
with open('data_columns.pkl', 'wb') as file:
    pickle.dump(data_columns, file)

In [3]:
import pickle

#Dataset only DICTrank
for dataset in ["DICTrank"]:
    
    # Exclude hidden files or directories like .ipynb_checkpoints
    if dataset.startswith('.'):
        continue
    print(dataset)

    # Get all the file names for this dataset
    all_files = os.listdir(os.path.join(data_path, dataset))

    # Extract activity names by removing the _train.csv.gz or _test.csv.gz from file names
    activity_names = list(set([f.replace("_train.csv.gz", "").replace("_test.csv.gz", "")  for f in all_files if not f.startswith(".ipynb_checkpoints")]))

    for activity in tqdm(activity_names, desc="Processing activities"):
        
        train_path = os.path.join(data_path, dataset, f"{activity}_train.csv.gz")
        test_path = os.path.join(data_path, dataset, f"{activity}_test.csv.gz")

        train_df = pd.read_csv(train_path, compression='gzip')
        test_df = pd.read_csv(test_path, compression='gzip')
        
        Ser_Mol_train = train_df['Standardized_SMILES'].apply(Chem.MolFromSmiles)
        Mordred_table_train = calc.pandas(Ser_Mol_train)
        Mordred_table_train = Mordred_table_train.astype('float')
               
        Ser_Mol_test = test_df['Standardized_SMILES'].apply(Chem.MolFromSmiles)
        Mordred_table_test = calc.pandas(Ser_Mol_test)
        Mordred_table_test = Mordred_table_test.astype('float')

        # Retain only those columns in the test dataset
        Mordred_table_train = Mordred_table_train[data_columns]
        Mordred_table_test = Mordred_table_test[data_columns]

        X_train = np.array(Mordred_table_train)
        X_test = np.array(Mordred_table_test)
        y_train = train_df[activity]
        y_test = test_df[activity]

      
        # Classification
        model = RandomForestClassifier(n_jobs=40)
            
        # Hyperparameter Optimization
        param_dist_classification = {'max_depth': randint(10, 20),
                          'max_features': randint(40, 50),
                          'min_samples_leaf': randint(5, 15),
                          'min_samples_split': randint(5, 15),
                          'n_estimators':[200, 300, 400, 500, 600],
                          'bootstrap': [True, False],
                          'oob_score': [False],
                          'random_state': [42],
                          'criterion': ['gini', 'entropy'],
                          'n_jobs': [40],
                          'class_weight' : [None, 'balanced']
                         }
        inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)   
            
        classification_search = HalvingRandomSearchCV(
                model,
                param_dist_classification,
                factor=3,
                cv=inner_cv,
                random_state=42,
                verbose=1,
                n_jobs=40)
            
        classification_search.fit(X_train, y_train)
        best_model = classification_search.best_estimator_
            
        # Random Over-sampling 
        sampler = RandomOverSampler(sampling_strategy='auto', random_state=42)
            
        pipeline = Pipeline(steps=[('sampler', sampler), ('model', best_model)])
        pipeline.fit(X_train, y_train)
        pickle.dump(pipeline, open("Physicochemical_model.sav",  'wb'))
        
        # Predict using threshold-optimized model
        probs_test = pipeline.predict_proba(X_test)[:, 1]
            
        oof_predictions = np.zeros(X_train.shape[0])
        oof_probs = np.zeros(X_train.shape[0])

        cv_scores = []

        for train_idx, valid_idx in inner_cv.split(X_train, y_train):
            X_train_fold, y_train_fold = X_train[train_idx], y_train[train_idx]
            X_valid_fold, y_valid_fold = X_train[valid_idx], y_train[valid_idx]

            # Random Over-sampling
            X_resampled, y_resampled = sampler.fit_resample(X_train_fold, y_train_fold)

            # Train the model on the resampled data
            best_model.fit(X_resampled, y_resampled)

            # Store out-of-fold predictions
            oof_predictions[valid_idx] = best_model.predict(X_valid_fold)
            oof_probs[valid_idx] = best_model.predict_proba(X_valid_fold)[:, 1]

            # AUC for this fold
            fold_auc = roc_auc_score(y_valid_fold, oof_probs[valid_idx])
            cv_scores.append(fold_auc)

        # Optimize the threshold using out-of-fold predictions
        best_threshold = optimize_threshold_j_statistic(y_train, oof_probs)
        predictions_test = (probs_test >= best_threshold).astype(int)

        results[activity] = {
                'CV_AUC_mean': np.mean(cv_scores),
                'CV_AUC_std': np.std(cv_scores),
                **evaluate_classifier(y_test, predictions_test, probs_test)
            }
        
        held_out_data = {
            'Dataset': dataset,
            "Actviity": activity,
            'SMILES': test_df['Standardized_SMILES'],
            'True_Value': y_test,
            'Prediction': predictions_test,
            'Probability': probs_test,
            'Best_Threshold': best_threshold
        }
        
        held_out_results.append(pd.DataFrame(held_out_data))         
        # Save results at each step
            
            
    # Save results at each step
    pd.DataFrame(results).T.to_csv('./physicochemical_model_results.csv')
              

# Save results
results_df = pd.DataFrame(results).T.reset_index(drop=False)
results_df = results_df.rename(columns={'index': 'endpoint'})
results_df.to_csv('./physicochemical_model_results.csv', index=False)

# Concatenate and save held-out test set results
pd.concat(held_out_results).to_csv('./physicochemical_model_held_out_test_results.csv', index=False)
            

DICTrank


  0%|                                           | 1/930 [00:00<05:52,  2.63it/s]
                                                                                
  3%|█▎                                        | 28/930 [00:01<01:01, 14.55it/s]
                                                                                
Processing activities:   0%|                              | 0/1 [00:01<?, ?it/s]
                                                                                
Processing activities:   0%|                              | 0/1 [00:01<?, ?it/s]
                                                                                
Processing activities:   0%|                              | 0/1 [00:01<?, ?it/s]
                                                                                
Processing activities:   0%|                              | 0/1 [00:01<?, ?it/s]
                                                                                
 14%|█████▋                 

/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, ax

                                                                                
 19%|███████▌                                | 177/930 [00:02<00:04, 165.46it/s]
                                                                                
 21%|████████▋                                | 196/930 [00:02<00:07, 92.28it/s]

/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


...

/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/mordred/_matrix_attributes.py:251: RuntimeWarning: invalid value encountered in double_scalars
  s += (eig.vec[i, eig.max] * eig.vec[j, eig.max]) ** -0.5
/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)



  4%|█▉                                          | 4/90 [00:00<00:06, 13.92it/s]
                                                                                
100%|██████████████████████████████████████████| 90/90 [00:00<00:00, 115.33it/s]

/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 20
max_resources_: 930
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 46
n_resources: 20
Fitting 5 folds for each of 46 candidates, totalling 230 fits


----------
iter: 1
n_candidates: 16
n_resources: 60
Fitting 5 folds for each of 16 candidates, totalling 80 fits
----------
iter: 2
n_candidates: 6
n_resources: 180
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 3
n_candidates: 2
n_resources: 540
Fitting 5 folds for each of 2 candidates, totalling 10 fits


Processing activities: 100%|██████████████████████| 1/1 [00:40<00:00, 40.21s/it]


In [4]:
held_out_results

[     Dataset  Actviity                                             SMILES  \
 0   DICTrank  DICTrank     O=c1n(CCC[NH+]2CCN(c3cccc(Cl)c3)CC2)nc2ccccn12   
 1   DICTrank  DICTrank                CC(C(=O)[O-])c1ccc(-c2ccccc2)c(F)c1   
 2   DICTrank  DICTrank                 C[NH+](C)CCC=C1c2ccccc2CCc2ccccc21   
 3   DICTrank  DICTrank  CC(=O)[NH+]1CCN(c2ccc(OCC3COC(Cn4ccnc4)(c4ccc(...   
 4   DICTrank  DICTrank                  Cc1nccn1CC1CCc2c(c3ccccc3n2C)C1=O   
 ..       ...       ...                                                ...   
 85  DICTrank  DICTrank                         CCC1(c2ccccc2)C(=O)NCNC1=O   
 86  DICTrank  DICTrank   CCOC(=O)[NH+]1CCC(=C2c3ccc(Cl)cc3CCc3cccnc32)CC1   
 87  DICTrank  DICTrank                 CCCSc1ccc2[n-]c(=NC(=O)OC)[n-]c2c1   
 88  DICTrank  DICTrank          CCC[NH+](CCC)S(=O)(=O)c1ccc(C(=O)[O-])cc1   
 89  DICTrank  DICTrank                 CC[NH+](CC)C(=S)SSC(=S)[NH+](CC)CC   
 
     True_Value  Prediction  Probability  Best_Threshold  
 0 

In [5]:
results_df

,endpoint,CV_AUC_mean,CV_AUC_std,Held_out_AUC,Held_out_AUCPR,Held_out_BA,Held_out_F1,Held_out_FN,Held_out_FP,Held_out_MCC,Held_out_Sensitivity,Held_out_Specificity,Held_out_TN,Held_out_TP
0,DICTrank,0.698334,0.064899,0.840615,0.932518,0.690769,0.842105,9.0,12.0,0.397648,0.861538,0.52,13.0,56.0


In [6]:
loaded_rf = pickle.load(open("Physicochemical_model.sav", 'rb'))

In [7]:
params = loaded_rf.get_params()


In [8]:
params

{'memory': None,
 'steps': [('sampler', RandomOverSampler(random_state=42)),
  ('model',
   RandomForestClassifier(bootstrap=False, class_weight='balanced', max_depth=18,
                          max_features=46, min_samples_leaf=5, min_samples_split=5,
                          n_estimators=200, n_jobs=40, random_state=42))],
 'verbose': False,
 'sampler': RandomOverSampler(random_state=42),
 'model': RandomForestClassifier(bootstrap=False, class_weight='balanced', max_depth=18,
                        max_features=46, min_samples_leaf=5, min_samples_split=5,
                        n_estimators=200, n_jobs=40, random_state=42),
 'sampler__random_state': 42,
 'sampler__sampling_strategy': 'auto',
 'sampler__shrinkage': None,
 'model__bootstrap': False,
 'model__ccp_alpha': 0.0,
 'model__class_weight': 'balanced',
 'model__criterion': 'gini',
 'model__max_depth': 18,
 'model__max_features': 46,
 'model__max_leaf_nodes': None,
 'model__max_samples': None,
 'model__min_impurity_decrease

In [9]:
classifier= RandomForestClassifier(bootstrap=False, class_weight='balanced', max_depth=18,
                          max_features=46, min_samples_leaf=5, min_samples_split=5,
                          n_estimators=200, n_jobs=40, random_state=42)

In [10]:
X_train

array([[ 19.77512879,  14.803335  ,   0.        , ..., 140.        ,
        172.        ,   5.        ],
       [ 23.80059255,  17.74158608,   0.        , ..., 154.        ,
        173.        ,   6.88888889],
       [  6.16336313,   6.51589884,   0.        , ...,  36.        ,
         36.        ,   2.11111111],
       ...,
       [  7.95651408,   7.58116421,   0.        , ...,  50.        ,
         55.        ,   2.61111111],
       [  7.38715648,   7.65713981,   0.        , ...,  50.        ,
         58.        ,   2.25      ],
       [  7.80668416,   7.34357946,   2.        , ...,  52.        ,
         61.        ,   2.22222222]])

In [11]:
y_train.shape

(930,)

In [12]:
X_test

array([[ 20.5631158 ,  14.95163401,   0.        , ..., 138.        ,
        162.        ,   5.69444444],
       [ 13.71082758,  11.69985029,   1.        , ...,  90.        ,
        104.        ,   4.02777778],
       [ 16.32047511,  13.12450461,   0.        , ..., 108.        ,
        126.        ,   4.69444444],
       ...,
       [ 13.50397853,  11.40209184,   2.        , ...,  88.        ,
        100.        ,   4.19444444],
       [ 13.80798646,  12.71610245,   1.        , ...,  90.        ,
        103.        ,   4.44444444],
       [ 10.74450109,  11.044544  ,   0.        , ...,  66.        ,
         72.        ,   4.13888889]])

In [13]:
y_test.shape

(90,)

In [14]:
X_all= np.vstack((X_train, X_test))
y_all= np.concatenate((y_train.to_numpy(), y_test.to_numpy()))

In [15]:
classifier.fit(X_all, y_all)

RandomForestClassifier(bootstrap=False, class_weight='balanced', max_depth=18,
                       max_features=46, min_samples_leaf=5, min_samples_split=5,
                       n_estimators=200, n_jobs=40, random_state=42)

In [16]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from numpy import argmax

inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)   
#Threshold Balancing
cross_val_prob_cp = cross_val_predict(classifier, X_all, y_all, cv=inner_cv, method='predict_proba', n_jobs=-1)[:, 1]
# calculate roc curves
fpr, tpr, thresholds = roc_curve(y_all, cross_val_prob_cp)
# get the best threshold
J = tpr - fpr
ix = argmax(J)
best_thresh_cp = thresholds[ix]
print('Best Threshold=%f' % (best_thresh_cp))

Best Threshold=0.641338


In [18]:
pickle.dump(classifier, open("FINAL_Physicochemical_model.sav",  'wb'))

In [19]:
df = pd.read_csv('./DICTrank_ambiguous_compounds.csv.gz')
df

,Unnamed: 0,Trade Name,Generic/Proper Name(s),Active Ingredient(s),SMILES,DICT _ Concern,Standardized_SMILES,Standardized_InChI
0,0,pancuronium bromide,pancuronium bromide,pancuronium bromide,[Br-].[Br-].[H][C@@]1(C[C@@]2([H])[C@]3([H])CC...,ambiguous,CC(=O)OC1CC2CCC3C(CCC4(C)C3CC([N+]3(C)CCCCC3)C...,InChI=1S/C35H60N2O4/c1-24(38)40-32-21-26-13-14...
1,1,vecuronium bromide,vecuronium bromide,vecuronium bromide,[Br-].CC(=O)O[C@H]1[C@H](C[C@H]2[C@@H]3CC[C@H]...,ambiguous,CC(=O)OC1CC2CCC3C(CCC4(C)C3CC([N+]3(C)CCCCC3)C...,InChI=1S/C34H57N2O4/c1-23(37)39-31-20-25-12-13...
2,2,doxercalciferol,doxercalciferol,doxercalciferol,[H][C@@]1(CC[C@@]2([H])\C(CCC[C@]12C)=C\C=C1\C...,ambiguous,C=C1C(=CC=C2CCCC3(C)C2CCC3C(C)C=CC(C)C(C)C)CC(...,InChI=1S/C28H44O2/c1-18(2)19(3)9-10-20(4)25-13...
3,3,"ergocalciferol capsules,","ergocalciferol capsules,",ergocalciferol,[H][C@@]1(CC[C@@]2([H])\C(CCC[C@]12C)=C\C=C1\C...,ambiguous,C=C1CCC(O)CC1=CC=C1CCCC2(C)C1CCC2C(C)C=CC(C)C(C)C,InChI=1S/C28H44O/c1-19(2)20(3)9-10-22(5)26-15-...
4,4,doxycycline,doxycycline,doxycycline,[H][C@@]12[C@@H](C)C3=C(C(O)=CC=C3)C(=O)C1=C(O...,ambiguous,CC1c2cccc([O-])c2C(=O)C2C(=O)C3(O)C(=O)C(C(N)=...,InChI=1S/C22H24N2O8/c1-7-8-5-4-6-9(25)11(8)16(...
...,...,...,...,...,...,...,...,...
80,80,chlorhexidine gluconate,chlorhexidine gluconate,chlorhexidine gluconate,OC[C@@H](O)[C@@H](O)[C@H](O)[C@@H](O)C(O)=O.OC...,ambiguous,NC(Nc1ccc(Cl)cc1)=[NH+]C(N)=[NH+]CCCCCC[NH2+]C...,InChI=1S/C22H30Cl2N10/c23-15-5-9-17(10-6-15)31...
81,81,dextrose,dextrose,dextrose,OC[C@@H](O)[C@@H](O)[C@H](O)[C@@H](O)C=O,ambiguous,O=C(C(O)CO)C(O)C(O)CO,InChI=1S/C6H12O6/c7-1-3(9)5(11)6(12)4(10)2-8/h...
82,82,didanosine,didanosine,didanosine,OC[C@@H]1CC[C@@H](O1)N1C=NC2=C1N=CNC2=O,ambiguous,O=c1[n-]cnc2c1ncn2C1CCC(CO)O1,InChI=1S/C10H12N4O3/c15-3-6-1-2-7(17-6)14-5-13...
83,83,oxazepam,oxazepam,oxazepam,OC1N=C(C2=CC=CC=C2)C2=CC(Cl)=CC=C2NC1=O,ambiguous,O=C1Nc2ccc(Cl)cc2C(c2ccccc2)NC1=O,InChI=1S/C15H11ClN2O2/c16-10-6-7-12-11(8-10)13...


In [20]:
Ser_Mol = df['Standardized_SMILES'].apply(Chem.MolFromSmiles)
Mordred_table = calc.pandas(Ser_Mol)
Mordred_table = Mordred_table.astype('float')
               
     

# Retain only those columns in the test dataset
Mordred_table = Mordred_table[data_columns]


X = np.array(Mordred_table)

100%|███████████████████████████████████████████| 85/85 [00:02<00:00, 31.47it/s]

/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, ax

In [21]:
probs_test = classifier.predict_proba(X)[:, 1]
probs_test

array([0.6098052 , 0.64652446, 0.86086053, 0.85906938, 0.52826605,
       0.65353698, 0.86113951, 0.40882933, 0.31185823, 0.86893012,
       0.70678552, 0.40335942, 0.5486982 , 0.63767118, 0.52569505,
       0.42983258, 0.92524706, 0.67135473, 0.63047089, 0.63878942,
       0.78590465, 0.85550038, 0.70678552, 0.56302615, 0.43313772,
       0.68195253, 0.64776009, 0.56399127, 0.64762543, 0.80746918,
       0.81003976, 0.4280809 , 0.6450672 , 0.69460967, 0.73588687,
       0.73519344, 0.71250945, 0.19255721, 0.50515711, 0.83105238,
       0.66725386, 0.47970961, 0.86124827, 0.71009297, 0.81956944,
       0.73139579, 0.63493314, 0.71964584, 0.69469649, 0.47625788,
       0.51838737, 0.69892285, 0.89434208, 0.37687966, 0.85596388,
       0.77991564, 0.45129883, 0.823815  , 0.57240143, 0.5016916 ,
       0.5789136 , 0.89638425, 0.96102839, 0.59688651, 0.77177992,
       0.73552547, 0.68937829, 0.54001462, 0.59688651, 0.76879109,
       0.69611543, 0.66271956, 0.78044473, 0.56799113, 0.72627

In [22]:
df["Probability"] = probs_test
df["Prediction"] = (probs_test >= best_threshold).astype(int)

In [23]:
df

,Unnamed: 0,Trade Name,Generic/Proper Name(s),Active Ingredient(s),SMILES,DICT _ Concern,Standardized_SMILES,Standardized_InChI,Probability,Prediction
0,0,pancuronium bromide,pancuronium bromide,pancuronium bromide,[Br-].[Br-].[H][C@@]1(C[C@@]2([H])[C@]3([H])CC...,ambiguous,CC(=O)OC1CC2CCC3C(CCC4(C)C3CC([N+]3(C)CCCCC3)C...,InChI=1S/C35H60N2O4/c1-24(38)40-32-21-26-13-14...,0.609805,0
1,1,vecuronium bromide,vecuronium bromide,vecuronium bromide,[Br-].CC(=O)O[C@H]1[C@H](C[C@H]2[C@@H]3CC[C@H]...,ambiguous,CC(=O)OC1CC2CCC3C(CCC4(C)C3CC([N+]3(C)CCCCC3)C...,InChI=1S/C34H57N2O4/c1-23(37)39-31-20-25-12-13...,0.646524,0
2,2,doxercalciferol,doxercalciferol,doxercalciferol,[H][C@@]1(CC[C@@]2([H])\C(CCC[C@]12C)=C\C=C1\C...,ambiguous,C=C1C(=CC=C2CCCC3(C)C2CCC3C(C)C=CC(C)C(C)C)CC(...,InChI=1S/C28H44O2/c1-18(2)19(3)9-10-20(4)25-13...,0.860861,1
3,3,"ergocalciferol capsules,","ergocalciferol capsules,",ergocalciferol,[H][C@@]1(CC[C@@]2([H])\C(CCC[C@]12C)=C\C=C1\C...,ambiguous,C=C1CCC(O)CC1=CC=C1CCCC2(C)C1CCC2C(C)C=CC(C)C(C)C,InChI=1S/C28H44O/c1-19(2)20(3)9-10-22(5)26-15-...,0.859069,1
4,4,doxycycline,doxycycline,doxycycline,[H][C@@]12[C@@H](C)C3=C(C(O)=CC=C3)C(=O)C1=C(O...,ambiguous,CC1c2cccc([O-])c2C(=O)C2C(=O)C3(O)C(=O)C(C(N)=...,InChI=1S/C22H24N2O8/c1-7-8-5-4-6-9(25)11(8)16(...,0.528266,0
...,...,...,...,...,...,...,...,...,...,...
80,80,chlorhexidine gluconate,chlorhexidine gluconate,chlorhexidine gluconate,OC[C@@H](O)[C@@H](O)[C@H](O)[C@@H](O)C(O)=O.OC...,ambiguous,NC(Nc1ccc(Cl)cc1)=[NH+]C(N)=[NH+]CCCCCC[NH2+]C...,InChI=1S/C22H30Cl2N10/c23-15-5-9-17(10-6-15)31...,0.699902,1
81,81,dextrose,dextrose,dextrose,OC[C@@H](O)[C@@H](O)[C@H](O)[C@@H](O)C=O,ambiguous,O=C(C(O)CO)C(O)C(O)CO,InChI=1S/C6H12O6/c7-1-3(9)5(11)6(12)4(10)2-8/h...,0.401882,0
82,82,didanosine,didanosine,didanosine,OC[C@@H]1CC[C@@H](O1)N1C=NC2=C1N=CNC2=O,ambiguous,O=c1[n-]cnc2c1ncn2C1CCC(CO)O1,InChI=1S/C10H12N4O3/c15-3-6-1-2-7(17-6)14-5-13...,0.630989,0
83,83,oxazepam,oxazepam,oxazepam,OC1N=C(C2=CC=CC=C2)C2=CC(Cl)=CC=C2NC1=O,ambiguous,O=C1Nc2ccc(Cl)cc2C(c2ccccc2)NC1=O,InChI=1S/C15H11ClN2O2/c16-10-6-7-12-11(8-10)13...,0.626587,0


In [24]:
df.to_csv('./DICTrank_ambiguous_compounds_predictions.csv', index=False)